In [1]:
import argparse
import json
import os
from pathlib import Path
from threading import Thread

import numpy as np
import torch
import yaml
from tqdm import tqdm

from models.experimental import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr
from utils.metrics import ap_per_class, ConfusionMatrix
from utils.plots import plot_images, output_to_target, plot_study_txt
from utils.torch_utils import select_device, time_synchronized, TracedModel

import argparse
import logging
import sys
from copy import deepcopy
import torch
import torch.nn as nn
sys.path.append('./')  # to run '$ python *.py' files in subdirectories
logger = logging.getLogger(__name__)
import torch
from models.common import *
from models.experimental import *
from utils.autoanchor import check_anchor_order
from utils.general import make_divisible, check_file, set_logging
from utils.torch_utils import time_synchronized, fuse_conv_and_bn, model_info, scale_img, initialize_weights, \
    select_device, copy_attr
from utils.loss import SigmoidBin
from models.yolo import *
try:
    import thop  # for FLOPS computation
except ImportError:
    thop = None


In [2]:

if __name__ == '__main__':
    parser = argparse.ArgumentParser(prog='test.py')
    parser.add_argument('--weights', nargs='+', type=str, default='yolov7.pt', help='model.pt path(s)')
    parser.add_argument('--data', type=str, default='data/coco.yaml', help='*.data path')
    parser.add_argument('--batch-size', type=int, default=32, help='size of each image batch')
    parser.add_argument('--img-size', type=int, default=640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=0.001, help='object confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.65, help='IOU threshold for NMS')
    parser.add_argument('--task', default='val', help='train, val, test, speed or study')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--single-cls', action='store_true', help='treat as single-class dataset')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--verbose', action='store_true', help='report mAP by class')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-hybrid', action='store_true', help='save label+prediction hybrid results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-json', action='store_true', help='save a cocoapi-compatible JSON results file')
    parser.add_argument('--project', default='runs/test', help='save to project/name')
    parser.add_argument('--name', default='exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--no-trace', action='store_true', help='don`t trace model')
    parser.add_argument('--v5-metric', action='store_true', help='assume maximum recall as 1.0 in AP calculation')
    opt = parser.parse_args([])
    opt.save_json |= opt.data.endswith('coco.yaml')
    opt.data = check_file(opt.data)  # check file
    print(opt)
    #check_requirements()


Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/coco.yaml', device='', exist_ok=False, img_size=640, iou_thres=0.65, name='exp', no_trace=False, project='runs/test', save_conf=False, save_hybrid=False, save_json=True, save_txt=False, single_cls=False, task='val', v5_metric=False, verbose=False, weights='yolov7.pt')


In [3]:
opt.batch_size

32

In [4]:
1, 3, opt.imgsz, opt.imgsz

AttributeError: 'Namespace' object has no attribute 'imgsz'

In [5]:
profile = True
y, dt = [], []  # outputs


device = select_device("cpu", batch_size=opt.batch_size)
model = attempt_load(opt.weights, map_location=device)  # load FP32 model
model.eval()

opt.imgsz=640
imgsz=640
x = torch.zeros(1, 3, opt.imgsz, opt.imgsz).to(device).type_as(next(model.parameters()))


## client inference


for m in model.model[:len(model.model)//2]:
    # print(m)
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

    if profile:
        c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
        o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
        for _ in range(10):
            m(x.copy() if c else x)
        t = time_synchronized()
        # for _ in range(10):
        #     m(x.copy() if c else x)
        m(x.copy() if c else x)
        
        dt.append((time_synchronized() - t) * 100)
        print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

    x = m(x)  # run
    y.append(x if m.i in model.save else None)  # save output
if profile:
    before_dt = sum(dt)
    print('before %.1fms total' % sum(dt))    



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
       0.0       928       2.1ms models.common.Conv                      
       0.0     18560       1.4ms models.common.Conv                      
       0.0     36992       1.5ms models.common.Conv                      
       0.0     73984       0.7ms models.common.Conv                      
       0.0      8320       0.1ms models.common.Conv                      
       0.0      8320       0.1ms models.common.Conv                      
       0.0     36992       0.3ms models.common.Conv                      
       0.0     36992       0.3ms models.common.Conv                      
       0.0     36992       0.3ms models.common.Conv                      
       0.0     36992       0.3ms models.common.Conv                      
       0.0         0       0.2ms models.common.Concat                    
       0.0     66048       1.0ms models.common.Conv                      
       0.0      

In [6]:

## server inference


device = select_device(opt.device, batch_size=opt.batch_size)
model = attempt_load(opt.weights, map_location=device)  # load FP32 model
model.eval()
for m in model.model[len(model.model)//2:]:
    # print(m)
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
    if isinstance(x, list):
        x = [t.to(device) for t in x]
    else:
        x = x.to(device)
    if profile:
        c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
        o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
        for _ in range(10):
            m(x.copy() if c else x)
        t = time_synchronized()
        # for _ in range(10):
        #     m(x.copy() if c else x)
        m(x.copy() if c else x)
        
        dt.append((time_synchronized() - t) * 100)
        print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

    x = m(x)  # run
    y.append(x if m.i in model.save else None)  # save output

if profile:
    print('after %.1fms total' % (sum(dt) - before_dt))
    # print('%.1fms total' % (sum(dt) - before_dt))

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
       0.0         0       0.0ms torch.nn.modules.upsampling.Upsample    
       0.0    262656       0.1ms models.common.Conv                      
       0.0         0       0.0ms models.common.Concat                    
       0.0    131584       0.1ms models.common.Conv                      
       0.0    131584       0.1ms models.common.Conv                      
       0.0    295168       0.1ms models.common.Conv                      
       0.0    147712       0.1ms models.common.Conv                      
       0.0    147712       0.1ms models.common.Conv                      
       0.0    147712       0.1ms models.common.Conv                      
       0.0         0       0.0ms models.common.Concat                    
       0.0    262656       0.1ms models.common.Conv                      
       0.0     33024       0.0ms models.common.Conv                      
       0.0      

/home/naver1/anaconda3/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
